In [ ]:
from pathlib import Path
from dummy_problems.dataloaders import LettersDataModule
from dummy_problems.models import MODEL_TYPES, SETTINGS_DL
import lightning as L

In [2]:
# INITIALISE DATAMODULE AND MODEL
settings =  {
    "model_type": "SVM",
    "model_name": "tiny_vit_21m_512.dist_in22k_ft_in1k",
    "dataset_dir": Path("/home/ubuntu/data/letters_dataset"),
    "stage": "train",
}
if settings["model_type"] == "DL":
    settings.update(SETTINGS_DL[settings["model_name"]])
    
data = LettersDataModule(settings)
model = MODEL_TYPES[settings['model_type']](settings)

In [ ]:
# TRAIN OR TEST BASED ON SETTINGS
if settings['model_type'] == "DL":
    if settings['stage'] == "fit":
        callbacks=[L.pytorch.callbacks.EarlyStopping(monitor="val_loss", mode="min")]
        trainer = L.Trainer(max_epochs=1000, callbacks=callbacks, log_every_n_steps=5)
        trainer.fit(model, data)
    elif settings['stage'] == "test":
        trainer = L.Trainer(log_every_n_steps=5)
        trainer.test(model=model, datamodule=data, ckpt_path=settings['checkpoint'])

elif settings['model_type'] == "SVM":
    data.setup("train")
    model.fit(data.letters_train)
    data.setup("test")
    model.test(data.letters_test)